In [ ]:
!pip install transformers

## ステップ1: 必要なライブラリのインストール

画像分類に必要な`transformers`ライブラリをインストールします。

## ステップ2: Google Driveの接続とフォルダ作成

Google Driveに接続して、判定したい画像を保存するフォルダを作成します。

## ステップ3: 画像分類モデルの読み込み

ここでは、**NSFW画像検出モデル**を読み込みます。このモデルは、画像が一般的な内容（normal）か、職場では不適切な内容（nsfw）かを判定します。

### 使用するモデル
- **モデル名**: Falconsai/nsfw_image_detection
- **アーキテクチャ**: Vision Transformer (ViT)
- **機能**: 2クラス分類（normal / nsfw）

# 深層学習体験会: 画像分類（コンテンツ判定）

このノートブックでは、**画像分類モデル**を使って、画像の内容を自動的に判定する方法を学びます。

## 画像分類とは？

画像分類とは、画像を見て、その画像がどのカテゴリーに属するかを判定するAI技術です。このノートブックでは、画像が一般的なコンテンツか、職場では不適切な内容（NSFW: Not Safe For Work）かを判定するモデルを使用します。

### 応用例
- SNSのコンテンツモデレーション
- 画像検索エンジンのフィルタリング
- オンラインプラットフォームの安全性確保
- 家族向けサービスのコンテンツ管理

## このノートブックの流れ
1. 必要なライブラリのインストール
2. Google Driveの接続
3. 画像分類モデルの読み込み
4. 画像の判定と結果の表示

## ステップ4: 画像の判定と結果の表示

Google Driveにアップロードした画像を読み込み、AIモデルで判定します。

### このコードの機能
1. **画像の読み込み**: 指定したパスから画像を読み込む
2. **推論の実行**: モデルが画像を判定
3. **色反転オプション**: `is_inverted`フラグで画像を反転表示できる
4. **結果の表示**: 判定結果（normal / nsfw）と画像を表示

### 使い方
1. Google Driveの`/MyDrive/images/`フォルダに判定したい画像をアップロード
2. 画像のファイル名を確認してコードを実行
3. `is_inverted = True`にすると、色を反転した画像が表示されます

In [ ]:
# Google Driveをマウント（接続）する
from google.colab import drive
import os

# Google Driveに接続（初回は認証が必要です）
drive.mount('/content/drive')

# 画像を保存するディレクトリのパス
images_dir = "/content/drive/MyDrive/images"

# ディレクトリが存在しない場合は作成する（exist_ok=Trueで既存の場合はエラーにしない）
os.makedirs(images_dir, exist_ok=True)

# 作成完了メッセージを表示
print("✅ フォルダ作成:", images_dir)

## 画像分類モデル

In [ ]:
# 必要なライブラリをインポート
import torch  # PyTorch（ディープラーニングフレームワーク）
from PIL import Image  # 画像処理用ライブラリ
from transformers import AutoModelForImageClassification, ViTImageProcessor  # Hugging Faceのモデル

# NSFW画像検出モデルを読み込む
# AutoModelForImageClassification: 画像分類用のモデルを自動で読み込む
# from_pretrained: 事前学習済みモデルをダウンロードして読み込む
model = AutoModelForImageClassification.from_pretrained("Falconsai/nsfw_image_detection")

# 画像プロセッサ（画像を前処理する）を読み込む
# ViTImageProcessor: Vision Transformer用の画像処理
processor = ViTImageProcessor.from_pretrained('Falconsai/nsfw_image_detection')

In [ ]:
# 必要なライブラリをインポート
from PIL import Image, ImageOps  # 画像処理用
import matplotlib.pyplot as plt  # グラフ・画像表示用

# 判定したい画像のパスを指定
img_path = "/content/drive/MyDrive/images/鬼滅_甘露寺さん.png"

# 画像を開いてRGB形式（カラー）に変換
img = Image.open(img_path).convert("RGB")

# --- 推論（AIモデルによる判定） ---
# torch.no_grad(): 学習モードではなく推論モードで実行（メモリ節約）
with torch.no_grad():
    # 画像をモデルが理解できる形式に変換
    # return_tensors="pt": PyTorch形式のテンソルとして返す
    inputs = processor(images=img, return_tensors="pt")
    
    # モデルに画像を入力して推論を実行
    outputs = model(**inputs)
    
    # 出力結果（logits）を取得
    logits = outputs.logits

# 最も確率が高いクラスのインデックスを取得
predicted_label = logits.argmax(-1).item()

# インデックスをラベル名（normal / nsfw）に変換
label = model.config.id2label[predicted_label]

# --- 色反転処理 ---
# 画像を反転表示するかどうかのフラグ
is_inverted = True  # ← Trueにすれば反転画像が表示される
if is_inverted:
    print("is_inverted がTrueなので色を反転して出力します")

# --- フラグに応じて画像を選択 ---
if is_inverted:
    # 色を反転させた画像を作成
    display_img = ImageOps.invert(img)
else:
    # 元の画像をそのまま使用
    display_img = img

# --- 結果表示 ---
# 判定結果に応じてメッセージを表示
if label == "normal":
    print("\n\n世間的に受け入れられる画像です")
else:
    print("⚠️ not suitable for work : 職場では不適当なコンテンツです")

# 画像を表示
plt.figure(figsize=(6,6))  # 図のサイズを指定
# タイトルに反転状態と判定結果を表示
plt.title(f"{'反転' if is_inverted else '元画像'} ({label})")
plt.imshow(display_img)  # 画像を表示
plt.axis("off")  # 軸を非表示
plt.show()  # グラフを表示